https://ui.adsabs.harvard.edu/abs/2019A%26A...621A.122B/abstract

In [ ]:
from astropy import units as u
distance = 2.2*u.kpc
field = 'G351.77'

In [ ]:
cd /orange/adamginsburg/salt/beuther/

In [ ]:
ls

for fn in glob.glob("*.fits"):
    print('app

In [ ]:
%matplotlib inline
import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'
pl.rcParams['font.size'] = 14
from spectral_cube import SpectralCube
from astropy import units as u
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
from astropy.table import Table
from astropy import table

In [ ]:
import pprint

In [ ]:
import scipy, scipy.ndimage

In [ ]:
import numpy as np
import os
import glob
from astropy import constants, units as u, table, stats, coordinates, wcs, log, coordinates as coord, visualization
import radio_beam
import pyspeckit
import pylab as pl
from astroquery.splatalogue.utils import minimize_table as mt
import sys
sys.path.append('/orange/adamginsburg/salt/Orion_ALMA_2016.1.00165.S/analysis')
import lines
import paths
from salt_tables import (salt_tables, salt_table_names, SO, SO2, HCl, sis_tables, AlCl, AlF, Al37Cl,
                         NaF, AlO, AlOH, NaCN, CaS, CaO)

In [ ]:
import regions
from astropy import units as u, coordinates
cutoutreg = reg = regions.Regions.read('../sanhueza/G351.77/cutout.reg')
disk1 = regions.Regions.read('disk1.reg') # mm1
disk2 = regions.Regions.read('disk2.reg') # mm2
disk3 = regions.Regions.read('disk3.reg') # mm12
#regvsm = regions.Regions.read('verysmall.reg')
#regvvsm = regions.Regions.read('innerdiskbox.reg')

In [ ]:
cont = SpectralCube.read('../sanhueza/G351.77/G351.77-0.54.config8.cont_avg.selfcal_0.5_hogbom.image.fits')
pl.figure(figsize=(10,10))
ax = pl.subplot(projection=cont[0].wcs)
ax.imshow(cont[0].value, cmap='gray_r', norm=visualization.simple_norm(cont[0].value, stretch='log'))
reg[0].to_pixel(cont[0].wcs).plot(ax=ax)
disk1[0].to_pixel(cont[0].wcs).plot(ax=ax)
disk2[0].to_pixel(cont[0].wcs).plot(ax=ax)
disk3[0].to_pixel(cont[0].wcs).plot(ax=ax)

pp = (reg[0] | disk3[0]).to_pixel(cont.wcs.celestial)
bb = pp.bounding_box
ax.axis([bb.ixmin-20, bb.ixmax+20, bb.iymin-20, bb.iymax+20])

# NaCl Stackl

In [ ]:
from spectral_cube.analysis_utilities import stack_cube

In [ ]:
nacltables = salt_tables[3:5]
#nacltables[0]

In [ ]:
import spectral_cube.analysis_utilities
import imp
imp.reload(spectral_cube.analysis_utilities)
from spectral_cube.analysis_utilities import stack_cube

In [ ]:
flist = (glob.glob('G351_spw[0123]_selfcal2.fits'))

In [ ]:
flist

In [ ]:
#from dask.diagnostics import ProgressBar
#pbar = ProgressBar()
#pbar.register()

import dask
dask.config.set(scheduler='threads', num_workers=8)

In [ ]:
for fn in flist:
    for rr,regn in ((disk1, 'disk1'), (disk2, 'disk2'), (disk3, 'disk3')):
        outfile = fn.replace(".fits",f"_{regn}.fits")
        if not os.path.exists(outfile) or (os.path.exists(outfile) and (SpectralCube.read(outfile).unit != u.K) or hasattr(SpectralCube.read(outfile), 'beams')):
            cube = SpectralCube.read(fn, use_dask=True).subcube_from_regions(rr)
            cube.allow_huge_operations=True
            if hasattr(cube, 'beams'):
                with cube.use_dask_scheduler('threads', num_workers=8):
                    cb = cube.beams.common_beam(max_iter=20, max_epsilon=0.01)
                    scube = cube.convolve_to(cb, save_to_tmp_dir=True)
            else:
                scube = cube
            assert not hasattr(scube, 'beams')
            scube.allow_huge_operations = True
            scube.to(u.K).write(outfile, overwrite=True)            

In [ ]:
regn = "disk1"
cubes = [SpectralCube.read(fn, use_dask=True).with_spectral_unit(u.GHz)
         for fn in (glob.glob(f'G351_spw[0123]_selfcal2_{regn}.fits'))]
cubes

In [ ]:
regn = "disk2"
cubesd2 = [SpectralCube.read(fn, use_dask=True)#.subcube_from_regions(reg3)
         for fn in (glob.glob(f'G351_spw[0123]_selfcal2_{regn}.fits'))]
cubesd2

In [ ]:
regn = "disk3"
cubesd3 = [SpectralCube.read(fn, use_dask=True)#.subcube_from_regions(reg3)
         for fn in (glob.glob(f'G351_spw[0123]_selfcal2_{regn}.fits'))]
cubesd3

In [ ]:
# only needed for VRSCs
if hasattr(cubes[0], 'beams'):
    beams = radio_beam.Beams(beams=[cube.beams.common_beam(max_iter=20, max_epsilon=0.01)
                                    if hasattr(cube, 'beams')
                                    else cube.beam
                                    for cube in cubes])
    cb=beams.common_beam()
else:
    beams = radio_beam.Beams(beams=[cube.beam for cube in cubes])
    cb=beams.common_beam()

In [ ]:
cb

In [ ]:
(cb.major*distance).to(u.au, u.dimensionless_angles())

In [ ]:
(cb.minor*distance).to(u.au, u.dimensionless_angles())

In [ ]:
linelist = np.concatenate([data['Freq'][(data['vu'] <= 3) &
                                          np.any([((data['Freq'].quantity > cube.with_spectral_unit(u.GHz).spectral_extrema[0]) &
                                                   (data['Freq'].quantity < cube.with_spectral_unit(u.GHz).spectral_extrema[1]))
                                                 for cube in cubes],
                                                 axis=0)].quantity
                             for data in nacltables])
linenames = np.concatenate( [data['Species'][(data['vu'] <= 3) &
                                          np.any([((data['Freq'].quantity > cube.with_spectral_unit(u.GHz).spectral_extrema[0]) &
                                                   (data['Freq'].quantity < cube.with_spectral_unit(u.GHz).spectral_extrema[1]))
                                                 for cube in cubes],
                                                 axis=0)]
                             for data in nacltables] )
(linelist, linenames)

In [ ]:
from astropy import log
log.setLevel('INFO')

In [ ]:
#log.setLevel('DEBUG')
vcen = -3.6 * u.km/u.s # from Patricio
stack, cutouts = stack_cube(cubes,
                   linelist=linelist,
                   vmin=vcen-35*u.km/u.s,
                   vmax=vcen+35*u.km/u.s,
                   average=np.nanmean,
                   convolve_beam=cb,
                   return_cutouts=True)
#log.setLevel('INFO')
stack.to(u.K).write('NaCl_v_le_3_stackcube_disk1.fits', overwrite=True)

In [ ]:
stackc = SpectralCube.read('NaCl_v_le_3_stackcube_disk1.fits', use_dask=True)
stackc

In [ ]:
#log.setLevel('DEBUG')
vcen = -3.6 * u.km/u.s # from Patricio
stack2, cutouts2 = stack_cube(cubesd2,
                   linelist=linelist,
                   vmin=vcen-25*u.km/u.s,
                   vmax=vcen+25*u.km/u.s,
                   average=np.nanmean,
                   convolve_beam=cb,
                   return_cutouts=True)
#log.setLevel('INFO')
stack2.to(u.K).write('NaCl_v_le_3_stackcube_disk2.fits', overwrite=True)

In [ ]:
#log.setLevel('DEBUG')
vcen3 = 0 * u.km/u.s # from Patricio
stack3, cutouts3 = stack_cube(cubesd3,
                   linelist=linelist,
                   vmin=vcen3-25*u.km/u.s,
                   vmax=vcen3+25*u.km/u.s,
                   average=np.nanmean,
                   convolve_beam=cb,
                   return_cutouts=True)
#log.setLevel('INFO')
stack3.to(u.K).write('NaCl_v_le_3_stackcube_disk3.fits', overwrite=True)

In [ ]:
pixscale_au = (np.abs((wcs.utils.proj_plane_pixel_area(stackc.wcs.celestial)**0.5*u.deg).to(u.arcsec)) * distance).to(u.au, u.dimensionless_angles())
pixscale_au

In [ ]:
m0 = stackc[:,:,:].moment0()

pl.imshow(m0.value, cmap='Blues', origin='lower',)# vmax=m0.value.max()*2,)
          #norm=simple_norm(m0.value, stretch='log', max_cut=m0.value.max()*10))
ax = pl.gca()

dp_60 = (60*u.au / pixscale_au).decompose()

pl.title("G351 mm1")
pl.plot([20, 20+dp_60], [25,25], color='r')
pl.text(20 + dp_60/2, 26, f'60 AU', horizontalalignment='center', color='r')
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))
axlims = ax.axis()

xc,yc = 18,18

xtlocs = ax.get_xticks()
xticklabels = map(lambda x: f"{x:0.2f}", (xtlocs-xc)*pixscale.value)
pl.xticks(xtlocs, xticklabels, rotation=45)

ytlocs = ax.get_yticks()
yticklabels = map(lambda x: f"{x:0.2f}", (ytlocs-yc)*pixscale.value)
pl.yticks(ytlocs, yticklabels, rotation=0)

ax.axis(axlims)

bm = stackc.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
pl.gca().add_artist(bmell)

offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)


ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}")
ax.set_ylabel("Dec offset (arcsec)")
pl.savefig("G351disk1_NaCl_mom0_ticks.png", bbox_inches='tight', dpi=300)

import reproject
contrep,_ = reproject.reproject_interp(cont[0].hdu, m0.header)
ax.contour(contrep, levels=[0.01, 0.02, 0.03, 0.04], colors=['orange']*5)
pl.savefig("G351disk1_NaCl_mom0_ticks_withcont.png", bbox_inches='tight', dpi=300)


In [ ]:
m0 = stack2[:,:,:].moment0()

pl.imshow(m0.value, cmap='Blues', origin='lower',)# vmax=m0.value.max()*2,)
          #norm=simple_norm(m0.value, stretch='log', max_cut=m0.value.max()*10))
ax = pl.gca()

pl.title("G351 mm2")
#pl.plot([13, 18], [18,18], color='r')
#pl.text(15.5, 19, f'{int(5*pixscale_au.value):0d} AU', horizontalalignment='center', color='r')
pl.plot([13, 13+dp_60], [18,18], color='r')
pl.text(13 + dp_60/2, 19, f'60 AU', horizontalalignment='center', color='r')
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))
axlims = ax.axis()

xc,yc = 12,12
xtlocs = ax.get_xticks()
xticklabels = map(lambda x: f"{x:0.2f}", (xtlocs-xc)*pixscale.value)
pl.xticks(xtlocs, xticklabels, rotation=45)

ytlocs = ax.get_yticks()
yticklabels = map(lambda x: f"{x:0.2f}", (ytlocs-yc)*pixscale.value)
pl.yticks(ytlocs, yticklabels, rotation=0)

ax.axis(axlims)

bm = stack2.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
pl.gca().add_artist(bmell)


offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)

ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}")
ax.set_ylabel("Dec offset (arcsec)")
pl.savefig("G351disk2_NaCl_mom0_ticks.png", bbox_inches='tight', dpi=300)

contrep,_ = reproject.reproject_interp(cont[0].hdu, m0.header)
ax.contour(contrep, levels=[0.01, 0.02, 0.03, 0.04], colors=['orange']*5)
pl.savefig("G351disk2_NaCl_mom0_ticks_withcont.png", bbox_inches='tight', dpi=300)

In [ ]:
m0 = stack3[:,:,:].subcube_from_regions(disk3).moment0()

pl.imshow(m0.value, cmap='Blues', origin='lower',)# vmax=m0.value.max()*2,)
          #norm=simple_norm(m0.value, stretch='log', max_cut=m0.value.max()*10))
ax = pl.gca()

pl.title("G351 mm12")
#pl.plot([13, 18], [18,18], color='r')
#pl.text(15.5, 19, f'{int(5*pixscale_au.value):0d} AU', horizontalalignment='center', color='r')
pl.plot([13, 13+dp_60], [18,18], color='r')
pl.text(13 + dp_60/2, 19, f'60 AU', horizontalalignment='center', color='r')
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))
axlims = ax.axis()

xc,yc = 12,12
xtlocs = ax.get_xticks()
xticklabels = map(lambda x: f"{x:0.2f}", (xtlocs-xc)*pixscale.value)
pl.xticks(xtlocs, xticklabels, rotation=45)

ytlocs = ax.get_yticks()
yticklabels = map(lambda x: f"{x:0.2f}", (ytlocs-yc)*pixscale.value)
pl.yticks(ytlocs, yticklabels, rotation=0)

ax.axis(axlims)

bm = stack2.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
pl.gca().add_artist(bmell)


offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)

ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}")
ax.set_ylabel("Dec offset (arcsec)")
pl.savefig("G351disk3_NaCl_mom0_ticks.png", bbox_inches='tight', dpi=300)

contrep,_ = reproject.reproject_interp(cont[0].hdu, m0.header)
ax.contour(contrep, levels=[0.01, 0.02, 0.03, 0.04], colors=['orange']*5)
pl.savefig("G351disk3_NaCl_mom0_ticks_withcont.png", bbox_inches='tight', dpi=300)

In [ ]:
# try creating a vmap based on another line (water)
restval = 232.6867*u.GHz # 'H2Ov2=1_5(5,0)-6(4,3)': 
cube = [c for c in cubes if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
print(cube)
h2ocube = watercube = scube = (cube.subcube_from_regions(disk1)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen, 20*u.km/u.s+vcen))

print(scube)
pl.figure(figsize=(14,5))
ax = pl.subplot(1,3,1)
m0 = scube.moment0()
ax.imshow(m0.value)
pl.title("m0")

ax = pl.subplot(1,3,2)
mx = scube.max(axis=0)
ax.imshow(mx.value)
pl.title("mx")
# no im pl.colorbar(mappable=im)
pl.subplot(1,3,3)
msk = mx > 150*u.K
msk = scipy.ndimage.binary_erosion(msk, iterations=1)
msk = scipy.ndimage.binary_dilation(msk, iterations=7)
pl.imshow(msk, interpolation='none', origin='lower')
pl.title('mask')
pl.tight_layout()
pl.figure()

from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (scube > 40*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m1h2omm1 = m1 = scube.with_mask(msk).with_mask(m3d).moment1()
m0h2omm1 = m0m = scube.with_mask(msk).with_mask(m3d).moment0()

pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1.value, vmin=vcen.value-10, vmax=vcen.value+10)
pl.colorbar(mappable=im)
mxv = scube.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
im = pl.imshow(mxv.value, vmin=vcen.value-10, vmax=vcen.value+10)
pl.colorbar(mappable=im)

pl.figure()
pl.imshow(m0m.value)
m0m.hdu.writeto('water_masked_moment0.fits', overwrite=True)

vmap = m1.hdu

In [ ]:
vmin, vmax = -15, 5

In [ ]:
from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (stackc > 40*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m0 = stackc.with_mask(m3d).moment0()
m1 = stackc.with_mask(m3d).moment1()

fig = pl.figure(figsize=(12,9))
ax1 = pl.subplot(2,2,1, )#projection=m0.wcs)
im1 = ax1.imshow(m0.value, cmap='Blues', origin='lower')
ax1.set_title("NaCl")
pl.colorbar(mappable=im1)


ax2 = pl.subplot(2,2,2, )#projection=m0h2omm1.wcs)
im2 = ax2.imshow(m0h2omm1.value, cmap='Blues', origin='lower',)
ax2.set_title("H$_2$O")
cb2 = pl.colorbar(mappable=im2)
cb2.set_label("Integrated Intensity [K km/s]")

ax3 = pl.subplot(2,2,3, )#projection=m1.wcs)
im3 = ax3.imshow(m1.value, cmap='RdBu_r', origin='lower',  vmin=vmin, vmax=vmax)
pl.colorbar(mappable=im3)


ax4 = pl.subplot(2,2,4, )#projection=m1h2omm1.wcs)
im4 = ax4.imshow(m1h2omm1.value, cmap='RdBu_r', origin='lower', vmin=vmin, vmax=vmax)
cb4 = pl.colorbar(mappable=im4)
cb4.set_label("$v_{LSR}$ [km/s]")

dp_60 = (60*u.au / pixscale_au).decompose()
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))

axlims = ax1.axis()

xc,yc = 16,14
offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)

for ax_ in (ax1,ax2,ax3,ax4):

    xtlocs = ax_.get_xticks()
    xticklabels = map(lambda x: f"{x:0.3f}", (xtlocs-xc)*pixscale.value)
    ax_.set_xticks(xtlocs, xticklabels, rotation=45)
    ax_.axhline(yc, color='k', linestyle='--', linewidth=0.5, alpha=0.5)
    ax_.axvline(xc, color='k', linestyle='--', linewidth=0.5, alpha=0.5)

    ytlocs = ax1.get_yticks()
    yticklabels = map(lambda x: f"{x:0.3f}", (ytlocs-yc)*pixscale.value)
    ax_.set_yticks(ytlocs, yticklabels, rotation=0)

    ax_.axis(axlims)


pl.suptitle("G351 mm1")
ax1.plot([20, 20+dp_60], [25,25], color='r')
ax1.text(20 + dp_60/2, 26, f'60 AU', horizontalalignment='center', color='r')

bm = stackc.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
ax1.add_artist(bmell)


ax = fig.add_subplot(111,frameon=False)
pl.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}", labelpad=15)
ax.set_ylabel("Dec offset (arcsec)", labelpad=15)


pl.tight_layout()
pl.savefig("G351mm1_m0m1_grid_ticks.png", bbox_inches='tight', dpi=300)

contrep,_ = reproject.reproject_interp(cont[0].hdu, m0.header)
for ax_ in (ax1,ax2,ax3,ax4):
    ax_.contour(contrep,
                levels=[0.01, 0.02, 0.03, 0.04], colors=['k']*5, linewidths=[0.5]*5)
pl.savefig("G351mm1_m0m1_grid_ticks_withcont.png", bbox_inches='tight', dpi=300)

In [ ]:
restval = 235.961352*u.GHz #SiS

cube = [c for c in cubes if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
siscube = (cube.subcube_from_regions(disk1)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen, 20*u.km/u.s+vcen))

restval = 234.935663*u.GHz # PN
cube = [c for c in cubes if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
pncube = (cube.subcube_from_regions(disk1)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen, 20*u.km/u.s+vcen))

In [ ]:
from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (siscube > 40*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m0sis = siscube.with_mask(m3d).moment0()
m1sis = siscube.with_mask(m3d).moment1()

m3d = (pncube > 40*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m0pn = pncube.with_mask(m3d).moment0()
m1pn = pncube.with_mask(m3d).moment1()

fig = pl.figure(figsize=(12,9))
ax1 = pl.subplot(2,2,1, )#projection=m0.wcs)
im1 = ax1.imshow(m0sis.value, cmap='Blues', origin='lower')
ax1.set_title("SiS")
pl.colorbar(mappable=im1)


ax2 = pl.subplot(2,2,2, )#projection=m0h2omm1.wcs)
im2 = ax2.imshow(m0pn.value, cmap='Blues', origin='lower',)
ax2.set_title("PN")
cb2 = pl.colorbar(mappable=im2)
cb2.set_label("Integrated Intensity [K km/s]")

ax3 = pl.subplot(2,2,3, )#projection=m1.wcs)
im3 = ax3.imshow(m1sis.value, cmap='RdBu_r', origin='lower', vmin=vmin, vmax=vmax)
pl.colorbar(mappable=im3)


ax4 = pl.subplot(2,2,4, )#projection=m1h2omm1.wcs)
im4 = ax4.imshow(m1pn.value, cmap='RdBu_r', origin='lower', vmin=vmin, vmax=vmax)
cb4 = pl.colorbar(mappable=im4)
cb4.set_label("$v_{LSR}$ [km/s]")

dp_60 = (60*u.au / pixscale_au).decompose()
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))

axlims = ax1.axis()

xc,yc = 16,14
offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)

for ax_ in (ax1,ax2,ax3,ax4):

    xtlocs = ax_.get_xticks()
    xticklabels = map(lambda x: f"{x:0.3f}", (xtlocs-xc)*pixscale.value)
    ax_.set_xticks(xtlocs, xticklabels, rotation=45)
    ax_.axhline(yc, color='k', linestyle='--', linewidth=0.5, alpha=0.5)
    ax_.axvline(xc, color='k', linestyle='--', linewidth=0.5, alpha=0.5)

    ytlocs = ax1.get_yticks()
    yticklabels = map(lambda x: f"{x:0.3f}", (ytlocs-yc)*pixscale.value)
    ax_.set_yticks(ytlocs, yticklabels, rotation=0)

    ax_.axis(axlims)


pl.suptitle("G351 mm1")
ax1.plot([20, 20+dp_60], [25,25], color='r')
ax1.text(20 + dp_60/2, 26, f'60 AU', horizontalalignment='center', color='r')

bm = stackc.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
ax1.add_artist(bmell)


ax = fig.add_subplot(111,frameon=False)
pl.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}", labelpad=20)
ax.set_ylabel("Dec offset (arcsec)", labelpad=15)


pl.tight_layout()
pl.savefig("G351mm1_m0m1_SiSPN_grid_ticks.png", bbox_inches='tight', dpi=300)

In [ ]:
#pl.imshow(m0h2omm1.value)
pl.figure(figsize=(12,8))

siocube = (SpectralCube.read('G351_spw1_selfcal2.fits', use_dask=True)
           .with_spectral_unit(u.GHz)
           .with_spectral_unit(u.km/u.s,
                               velocity_convention='radio',
                               rest_value=217.10498*u.GHz)
           .spectral_slab(vcen-40*u.km/u.s, vcen+40*u.km/u.s)
           .subcube_from_regions(cutoutreg)
          )
siocube.beam_threshold = 1
m3d = (siocube > 1.2*u.mJy/u.beam).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=4)
m3d = m3d.compute()

ax1 = pl.subplot(2,2,1, projection=siocube.wcs.celestial)
ax1.imshow(siocube.moment0(axis=0).value)
ax2 = pl.subplot(2,2,2, projection=siocube.wcs.celestial)
ax2.imshow(siocube.max(axis=0).value,)
ax3 = pl.subplot(2,2,3, projection=siocube.wcs.celestial)
ax3.imshow(siocube.with_mask(m3d).argmax_world(axis=0).value, vmin=-20, vmax=20, cmap='RdBu_r')
ax4 = pl.subplot(2,2,4, projection=siocube.wcs.celestial)
im4=ax4.imshow(siocube.with_mask(m3d).moment1(axis=0).value, vmin=-20, vmax=20, cmap='RdBu_r')
pl.colorbar(mappable=im4)
#redsiomm1 = siocube.spectral_slab(-50*u.km/u.s, -5*u.km/u.s).moment0(axis=0)
#bluesiomm1 = siocube.spectral_slab(-5*u.km/u.s, 40*u.km/u.s).moment0(axis=0)


pp = disk1[0].to_pixel(ax1.wcs.celestial)
bb = pp.bounding_box
for ax in (ax1,ax2,ax3,ax4):
    axlims = ax.axis()
    ax.contour(cont[0].value, transform=ax.get_transform(cont[0].wcs),
           levels=[0.01,0.02,0.03,0.04], colors=['k']*5, linewidths=[0.5]*5)
    ax.axis(axlims)
    ax.axis([bb.ixmin-50, bb.ixmax+50, bb.iymin-50, bb.iymax+50])
    

In [ ]:
#pl.imshow(m0h2omm1.value)
pl.figure(figsize=(12,8))

siocube2 = (SpectralCube.read('../sanhueza/G351.77/G351.77-0.54.config8.spw2.robust0.5.image_small.fits', use_dask=True)
           .with_spectral_unit(u.GHz)
           .with_spectral_unit(u.km/u.s,
                               velocity_convention='radio',
                               rest_value=217.10498*u.GHz)
           .spectral_slab(vcen-40*u.km/u.s, vcen+40*u.km/u.s)
           .subcube_from_regions(cutoutreg)
          )
siocube2.beam_threshold = 1
m3d = (siocube2 > 7*u.mJy/u.beam).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=4)
m3d = m3d.compute()

ax1 = pl.subplot(2,2,1, projection=siocube2.wcs.celestial)
ax1.imshow(siocube2.moment0(axis=0).value)
ax2 = pl.subplot(2,2,2, projection=siocube2.wcs.celestial)
ax2.imshow(siocube2.max(axis=0).value,)
ax3 = pl.subplot(2,2,3, projection=siocube2.wcs.celestial)
ax3.imshow(siocube2.with_mask(m3d).argmax_world(axis=0).value, vmin=-20, vmax=20, cmap='RdBu_r')
ax4 = pl.subplot(2,2,4, projection=siocube2.wcs.celestial)
im4=ax4.imshow(siocube2.with_mask(m3d).moment1(axis=0).value, vmin=-20, vmax=20, cmap='RdBu_r')
pl.colorbar(mappable=im4)
#redsiomm1 = siocube.spectral_slab(-50*u.km/u.s, -5*u.km/u.s).moment0(axis=0)
#bluesiomm1 = siocube.spectral_slab(-5*u.km/u.s, 40*u.km/u.s).moment0(axis=0)


pp = disk1[0].to_pixel(ax1.wcs.celestial)
bb = pp.bounding_box
for ax in (ax1,ax2,ax3,ax4):
    axlims = ax.axis()

    ax.contour(cont[0].value, transform=ax.get_transform(cont[0].wcs),
               levels=[0.01,0.02,0.03,0.04], colors=['k']*5, linewidths=[0.5]*5)
    ax.axis(axlims)
    ax.axis([bb.ixmin-50, bb.ixmax+50, bb.iymin-50, bb.iymax+50])
    

In [ ]:
vcen

In [ ]:
#pl.imshow(m0h2omm1.value)
pl.figure(figsize=(12,8))

siocube3 = (SpectralCube.read('/orange/adamginsburg/ALMAIMF_Images/G351.77/B6/linecubes_12m/G351.77_B6_spw1_12M_sio.JvM.image.pbcor.statcont.contsub.fits', use_dask=True)
           .with_spectral_unit(u.GHz)
           .with_spectral_unit(u.km/u.s,
                               velocity_convention='radio',
                               rest_value=217.10498*u.GHz)
           .spectral_slab(vcen-40*u.km/u.s, vcen+40*u.km/u.s)
           .subcube_from_regions(cutoutreg)
          )
siocube3.beam_threshold = 1
m3d = (siocube3 > 50*u.mJy/u.beam).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=4)
m3d = m3d.compute()

ax1 = pl.subplot(2,2,1, projection=siocube3.wcs.celestial)
ax1.imshow(siocube3.moment0(axis=0).value)
ax2 = pl.subplot(2,2,2, projection=siocube3.wcs.celestial)
ax2.imshow(siocube3.max(axis=0).value,)
ax3 = pl.subplot(2,2,3, projection=siocube3.wcs.celestial)
ax3.imshow(siocube3.with_mask(m3d).argmax_world(axis=0).value, vmin=-20, vmax=20, cmap='RdBu_r')
ax4 = pl.subplot(2,2,4, projection=siocube3.wcs.celestial)
im4=ax4.imshow(siocube3.with_mask(m3d).moment1(axis=0).value, vmin=-20, vmax=20, cmap='RdBu_r')
pl.colorbar(mappable=im4)
#redsiomm1 = siocube.spectral_slab(-50*u.km/u.s, -5*u.km/u.s).moment0(axis=0)
#bluesiomm1 = siocube.spectral_slab(-5*u.km/u.s, 40*u.km/u.s).moment0(axis=0)


#pp = disk1[0].to_pixel(ax1.wcs.celestial)
#bb = pp.bounding_box
for ax in (ax1,ax2,ax3,ax4):
#    ax.axis([bb.ixmin-50, bb.ixmax+50, bb.iymin-50, bb.iymax+50])
    axlims = ax.axis()
    ax.contour(cont[0].value, transform=ax.get_transform(cont[0].wcs),
               levels=[0.01,0.02,0.03,0.04], colors=['k']*5, linewidths=[0.5]*5)
    ax.axis(axlims)

In [ ]:
#siopv
import pvextractor
from astropy.coordinates import SkyCoord
ep1 = SkyCoord('17:26:42.2727978486 -36:09:14.5524690573', frame='icrs', unit=(u.h, u.deg))
ep2 = SkyCoord('17:26:42.7644124960 -36:09:19.8361908428', frame='icrs', unit=(u.h, u.deg))
siopath = pvextractor.Path(SkyCoord([ep1, ep2]), width=0.1*u.arcsec)

In [ ]:
siocube = (SpectralCube.read('G351_spw1_selfcal2.fits', use_dask=True)
           .with_spectral_unit(u.GHz)
           .with_spectral_unit(u.km/u.s,
                               velocity_convention='radio',
                               rest_value=217.10498*u.GHz)
           .spectral_slab(vcen-70*u.km/u.s, vcen+70*u.km/u.s)
          )

siocube2 = (SpectralCube.read('../sanhueza/G351.77/G351.77-0.54.config8.spw2.robust0.5.image_small.fits', use_dask=True)
           .with_spectral_unit(u.GHz)
           .with_spectral_unit(u.km/u.s,
                               velocity_convention='radio',
                               rest_value=217.10498*u.GHz)
           .spectral_slab(vcen-70*u.km/u.s, vcen+70*u.km/u.s)
          )
siocube3 = (SpectralCube.read('/orange/adamginsburg/ALMAIMF_Images/G351.77/B6/linecubes_12m/G351.77_B6_spw1_12M_sio.JvM.image.pbcor.statcont.contsub.fits', use_dask=True)
           .with_spectral_unit(u.GHz)
           .with_spectral_unit(u.km/u.s,
                               velocity_convention='radio',
                               rest_value=217.10498*u.GHz)
           .spectral_slab(vcen-70*u.km/u.s, vcen+70*u.km/u.s)
          )

pv1 = pvextractor.extract_pv_slice(siocube, siopath)
pv2 = pvextractor.extract_pv_slice(siocube2, siopath)
pv3 = pvextractor.extract_pv_slice(siocube3, siopath)

In [ ]:
from astropy.visualization import simple_norm

In [ ]:
pl.figure(figsize=(15,8))
pl.subplot(1,3,1, projection=wcs.WCS(pv1.header)).imshow(pv1.data,
                                                         origin='lower',
                                                         norm=simple_norm(pv1.data, max_percent=99.5, min_percent=1))
pl.gca().set_aspect(pv1.data.shape[1]/pv1.data.shape[0])

pl.subplot(1,3,2, projection=wcs.WCS(pv2.header)).imshow(pv2.data,
                                                         origin='lower',
                                                         norm=simple_norm(pv2.data, max_percent=99.9, min_percent=1))
pl.gca().set_aspect(pv2.data.shape[1]/pv2.data.shape[0])
pl.gca().set_ylabel("")
pl.subplot(1,3,3, projection=wcs.WCS(pv3.header)).imshow(pv3.data,
                                                         origin='lower',
                                                         norm=simple_norm(pv3.data, max_percent=99.9, min_percent=1))
pl.gca().set_aspect(pv3.data.shape[1]/pv3.data.shape[0])
pl.tight_layout()

for ax in pl.gcf().axes:
    ax.coords[1].set_format_unit(u.km/u.s)
    ax.coords[0].set_format_unit(u.arcsec)
    ax.coords[0].set_major_formatter('x.xx')
for ax in pl.gcf().axes[1:]:
    ax.coords[1].set_ticklabel_visible(False)
    ax.coords[1].set_axislabel("")


In [ ]:
pl.figure(figsize=(15,8))
pl.subplot(1,3,1, projection=wcs.WCS(pv1.header)).imshow(pv1.data,
                                                         origin='lower',
                                                         norm=simple_norm(pv1.data, max_percent=99.9, min_percent=1))
pl.gca().set_aspect(pv1.data.shape[1]/pv1.data.shape[0])

pl.subplot(1,3,2, projection=wcs.WCS(pv2.header)).imshow(pv2.data,
                                                         origin='lower',
                                                         norm=simple_norm(pv2.data, max_percent=99.9, min_percent=1))
pl.gca().set_aspect(pv2.data.shape[1]/pv2.data.shape[0])
pl.gca().set_ylabel("")
pl.subplot(1,3,3, projection=wcs.WCS(pv3.header)).imshow(pv3.data,
                                                         origin='lower',
                                                         norm=simple_norm(pv3.data, max_percent=99.9, min_percent=1))
pl.gca().set_aspect(pv3.data.shape[1]/pv3.data.shape[0])
pl.tight_layout()

for ax in pl.gcf().axes:
    ax.coords[1].set_format_unit(u.km/u.s)
    ax.coords[0].set_format_unit(u.arcsec)
    ax.coords[0].set_major_formatter('x.xx')
    ax.set_xlim(ax.wcs[0].world_to_pixel(3.5*u.arcsec),
                ax.wcs[0].world_to_pixel(4.5*u.arcsec))
    ax.set_aspect(ax.get_aspect()/5)
for ax in pl.gcf().axes[1:]:
    ax.coords[1].set_ticklabel_visible(False)
    ax.coords[1].set_axislabel("")

pl.tight_layout()

In [ ]:
offset_loc.to_string('hmsdms')

In [ ]:
# try creating a vmap based on another line (water)
restval = 232.6867*u.GHz # 'H2Ov2=1_5(5,0)-6(4,3)': 
vcen2 = -6*u.km/u.s
cube = [c for c in cubesd2 if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
print(cube)
scube = (cube.subcube_from_regions(disk2)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen2, 20*u.km/u.s+vcen2))
print(scube)
pl.figure(figsize=(14,5))
ax = pl.subplot(1,3,1)
m0 = scube.moment0()
ax.imshow(m0.value)
pl.title("m0")

ax = pl.subplot(1,3,2)
mx = scube.max(axis=0)
im = ax.imshow(mx.value)
pl.title("mx")
pl.colorbar(mappable=im)
pl.subplot(1,3,3)
msk = mx > 50*u.K
msk = scipy.ndimage.binary_erosion(msk, iterations=2)
msk = scipy.ndimage.binary_dilation(msk, iterations=7)
pl.imshow(msk, interpolation='none', origin='lower')
pl.title('mask')
pl.tight_layout()
pl.figure()

from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (scube > 25*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m1 = scube.with_mask(msk).with_mask(m3d).moment1()
m0m = scube.with_mask(msk).with_mask(m3d).moment0()
pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1.value, vmin=vcen2.value-15, vmax=vcen2.value+15)
pl.colorbar(mappable=im)
mxv = scube.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
im = pl.imshow(mxv.value, vmin=vcen2.value-15, vmax=vcen2.value+15)
pl.colorbar(mappable=im)


pl.figure()
pl.imshow(m0m.value)

vmap2 = m1.hdu

In [ ]:
# try creating a vmap based on another line (water)
restval = 232.6867*u.GHz # 'H2Ov2=1_5(5,0)-6(4,3)': 
vcen3 = 0*u.km/u.s
cube = [c for c in cubesd3 if (c.with_spectral_unit(u.GHz).spectral_extrema[0] < restval) & (c.with_spectral_unit(u.GHz).spectral_extrema[1] > restval)][0]
print(cube)
scube = (cube.subcube_from_regions(disk3)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen3, 20*u.km/u.s+vcen3))
print(scube)
pl.figure(figsize=(14,5))
ax = pl.subplot(1,3,1)
m0 = scube.moment0()
ax.imshow(m0.value)
pl.title("m0")

ax = pl.subplot(1,3,2)
mx = scube.max(axis=0)
im = ax.imshow(mx.value)
pl.title("mx")
pl.colorbar(mappable=im)
pl.subplot(1,3,3)
msk = mx > 50*u.K
msk = scipy.ndimage.binary_erosion(msk, iterations=2)
msk = scipy.ndimage.binary_dilation(msk, iterations=7)
pl.imshow(msk, interpolation='none', origin='lower')
pl.title('mask')
pl.tight_layout()
pl.figure()

from dask_image.ndmorph import binary_dilation, binary_erosion
m3d = (scube > 25*u.K).include()
m3d = binary_erosion(m3d, iterations=1)
m3d = binary_dilation(m3d, iterations=2)
m3d = m3d.compute()

m1h2omm12 = scube.with_mask(msk).with_mask(m3d).moment1()
m0mh2omm12 = scube.with_mask(msk).with_mask(m3d).moment0()
pl.figure(figsize=(14,6))
ax1 = pl.subplot(1,2,1)
im = ax1.imshow(m1h2omm12.value, vmin=vcen3.value-10, vmax=vcen3.value+10)
pl.colorbar(mappable=im)
mxv = scube.with_mask(msk).argmax_world(axis=0)
pl.subplot(1,2,2)
im = pl.imshow(mxv.value, vmin=vcen3.value-10, vmax=vcen3.value+10)
pl.colorbar(mappable=im)


pl.figure()
pl.imshow(m0m.value)

vmap3 = m1.hdu

In [ ]:
from dask_image.ndmorph import binary_dilation, binary_erosion

slcs = slice(None), slice(4,-1), slice(6,-6)
m3d = (stack3.subcube_from_regions(disk3) > 40*u.K).include()
m3d = scipy.ndimage.binary_erosion(m3d, iterations=1)
m3d = scipy.ndimage.binary_dilation(m3d, iterations=2)
#m3d = m3d.compute()

m0 = stack3.subcube_from_regions(disk3).with_mask(m3d)[slcs].moment0()
m1 = stack3.subcube_from_regions(disk3).with_mask(m3d)[slcs].moment1()


h2ocube3 = watercube3 = scube3 = (cube.subcube_from_regions(disk3)
         .with_spectral_unit(u.GHz)
         .with_spectral_unit(u.km/u.s,
                             velocity_convention='radio',
                             rest_value=restval)
         .spectral_slab(-20*u.km/u.s+vcen3, 20*u.km/u.s+vcen3))

m3dw = (h2ocube3 > 40*u.K).include()
m3dw = binary_erosion(m3dw, iterations=1)
m3dw = binary_dilation(m3dw, iterations=2)
m3dw = m3dw.compute()

m1h2omm12 = scube3.with_mask(m3dw)[slcs].moment1()
m0h2omm12 = scube3.with_mask(m3dw)[slcs].moment0()

fig = pl.figure(figsize=(12,9))
ax1 = pl.subplot(2,2,1, )#projection=m0.wcs)
im1 = ax1.imshow(m0.value, cmap='Blues', origin='lower')
ax1.set_title("NaCl")
pl.colorbar(mappable=im1)


ax2 = pl.subplot(2,2,2, )#projection=m0h2omm1.wcs)
im2 = ax2.imshow(m0h2omm12.value, cmap='Blues', origin='lower',)
ax2.set_title("H$_2$O")
cb2 = pl.colorbar(mappable=im2)
cb2.set_label("Integrated Intensity [K km/s]")

ax3 = pl.subplot(2,2,3, )#projection=m1.wcs)
im3 = ax3.imshow(m1.value, cmap='RdBu_r', origin='lower',  vmin=vcen3.value-15, vmax=vcen3.value+15)
pl.colorbar(mappable=im3)


ax4 = pl.subplot(2,2,4, )#projection=m1h2omm1.wcs)
im4 = ax4.imshow(m1h2omm12.value, cmap='RdBu_r', origin='lower', vmin=vcen3.value-15, vmax=vcen3.value+15)
cb4 = pl.colorbar(mappable=im4)
cb4.set_label("$v_{LSR}$ [km/s]")

dp_60 = (60*u.au / pixscale_au).decompose()
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))

axlims = ax1.axis()

xc,yc = 13,10
offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)
#offset_loc = disk3[0].center
#offset_loc = coordinates.SkyCoord("17:26:42.344 -36:09:16.14", unit=(u.h, u.deg), frame='icrs')
#xc,yc = m0.wcs.celestial.world_to_pixel(offset_loc)

for ax_ in (ax1,ax2,ax3,ax4):

    xtlocs = ax_.get_xticks()
    xticklabels = map(lambda x: f"{x:0.3f}", (xtlocs-xc)*pixscale.value)
    ax_.set_xticks(xtlocs, xticklabels, rotation=45)
    ax_.axhline(yc, color='k', linestyle='--', linewidth=0.5, alpha=0.5)
    ax_.axvline(xc, color='k', linestyle='--', linewidth=0.5, alpha=0.5)

    ytlocs = ax1.get_yticks()
    yticklabels = map(lambda x: f"{x:0.3f}", (ytlocs-yc)*pixscale.value)
    ax_.set_yticks(ytlocs, yticklabels, rotation=0)

    ax_.axis(axlims)


pl.suptitle("G351 mm12")
blx = xc
bly = 2
ax1.plot([blx, blx+dp_60], [bly,bly], color='r')
ax1.text(blx + dp_60/2, bly+1, f'60 AU', horizontalalignment='center', color='r')

bm = stackc.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
ax1.add_artist(bmell)


ax = fig.add_subplot(111,frameon=False)
pl.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}", labelpad=15)
ax.set_ylabel("Dec offset (arcsec)", labelpad=15)


pl.tight_layout()
pl.savefig("G351mm12_m0m1_grid_ticks.png", bbox_inches='tight', dpi=300)

contrep,_ = reproject.reproject_interp(cont[0].hdu, m0.header)
for ax_ in (ax1,ax2,ax3,ax4):
    ax_.contour(contrep,
                levels=[0.01, 0.02, 0.03, 0.04], colors=['k']*5, linewidths=[0.5]*5)
pl.savefig("G351mm12_m0m1_grid_ticks_withcont.png", bbox_inches='tight', dpi=300)

In [ ]:
import sys
sys.path.append('/orange/adamginsburg/salt/code/')
import imp, overplot_saltlines; imp.reload(overplot_saltlines)
from overplot_saltlines import overplot_saltlines

In [ ]:
import imp
import overplot_saltlines
imp.reload(overplot_saltlines)
import lines
imp.reload(lines)
from overplot_saltlines import overplot_saltlines, all_lines, linetexnames, linefreqs
'SO65-54' in all_lines, 'H30a' in all_lines, 'H30$\\alpha$' in linetexnames

In [ ]:
spectra = []
for ii,cube in enumerate(cubes):
    cube.allow_huge_operations=True
    sp = cube.subcube_from_regions(disk1).with_spectral_unit(u.GHz).mean(axis=(1,2))
    pysp = pyspeckit.Spectrum(data=sp, xarr=sp.spectral_axis, header=sp.header)
    spectra.append(pysp)
    pysp.specname = f'G351.77 mm1 spw{ii+1}'

In [ ]:
spectra2 = []
for ii,cube in enumerate(cubesd2):
    cube.allow_huge_operations=True
    sp = cube.subcube_from_regions(disk2).with_spectral_unit(u.GHz).mean(axis=(1,2))
    pysp = pyspeckit.Spectrum(data=sp, xarr=sp.spectral_axis, header=sp.header)
    spectra2.append(pysp)
    pysp.specname = f'G351.77 mm2 spw{ii+1}'

In [ ]:
spectra3 = []
for ii,cube in enumerate(cubesd3):
    cube.allow_huge_operations=True
    sp = cube.subcube_from_regions(disk3).with_spectral_unit(u.GHz).mean(axis=(1,2))
    pysp = pyspeckit.Spectrum(data=sp, xarr=sp.spectral_axis, header=sp.header)
    spectra3.append(pysp)
    pysp.specname = f'G351.77 mm12 spw{ii+1}'

In [ ]:
mkdir -p spectra/figures/disk1

In [ ]:
mkdir -p spectra/figures/disk2

In [ ]:
mkdir -p spectra/figures/disk3

In [ ]:
overplot_saltlines(spectra, vcen=vcen, savepath='spectra/figures/disk1', ymax=180, ymin=-25, yfrac=0.6)

In [ ]:
overplot_saltlines(spectra2, vcen=vcen2, savepath='spectra/figures/disk2', ymax=180, ymin=-25, yfrac=0.55)

In [ ]:
vcen

In [ ]:
overplot_saltlines(spectra3, vcen=vcen3, savepath='spectra/figures/disk3', ymax=100, ymin=-15, yfrac=0.55)

In [ ]:
from IPython.display import Image,display

In [ ]:
for fn in glob.glob('spectra/figures/disk1/color*png'):
    display(Image(fn))

In [ ]:
pwd

In [ ]:
ls $PWD/spectra/figures/disk2/color*png

In [ ]:
for fn in glob.glob('spectra/figures/disk2/color*png'):
    display(Image(fn))

In [ ]:
for fn in glob.glob('spectra/figures/disk3/color*png'):
    display(Image(fn))

In [ ]:
name_mapping = {'disk1': 'G351mm1', 'disk2': 'G351mm2', 'disk3': 'G351mm3'}

In [ ]:
import glob
import spectral_cube.analysis_utilities
import reproject
from tqdm.notebook import tqdm
from astropy.io import fits

for reg, cubelist, basename, vm in zip((disk1, disk2, disk3),
                                       (cubes, cubesd2, cubesd3),
                                       ('disk1', 'disk2', 'disk3',),
                                       (vmap, vmap2, vmap3)):
    print(basename)
    for spwn,cube in enumerate(cubelist):
        print(cube)
        scube = cube.subcube_from_regions(reg).to(u.K)
        # continuum subtract before stacking
        #scube = scube - scube.percentile(25, axis=0)
        #mx = scube.max(axis=0)
        #msk = mx > 0.003*mx.unit
        print("Mean spectrum")
        sp = scube.mean(axis=(1,2))

        #sp.write(f'spectra/{fn.split("/")[-1].replace(".fits","_meanspec.fits")}', overwrite=True)


        # reproject the velocity map into the cube's coordinate system
        vmap_proj,_ = reproject.reproject_interp(vm,
                                                 scube.wcs.celestial,
                                                 shape_out=scube.shape[1:])
        fits.PrimaryHDU(data=vmap_proj, header=scube.wcs.celestial.to_header()).writeto(f'vmap_{basename}.fits', overwrite=True)
        vmap_proj = u.Quantity(vmap_proj, u.km/u.s)
        
        # convert to km/s with arbitrary rest value so 0 km/s can be interpreted
        if scube.spectral_axis.unit.is_equivalent(u.Hz):
            scube = scube.with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=scube.spectral_axis.mean())

        print("Stacking")
        # perform the stacking!
        stack = spectral_cube.analysis_utilities.stack_spectra(scube, vmap_proj,
                                                               v0=0.0*u.km/u.s,
                                                               #progressbar=tqdm,
                                                               #num_cores=8
                                                              )
        fstack = stack.with_spectral_unit(u.GHz)

        pl.suptitle(f"{fn}")
        fstack.write(f'spectra/{basename}_{spwn}_{name_mapping[basename]}_stack.fits', overwrite=True)
        fstack.write(f'spectra/{name_mapping[basename]}_{spwn}_{name_mapping[basename]}_stack.fits', overwrite=True)
        

        pl.clf()
        fstack.quicklook(filename=f'spectra/figures/{basename}_{spwn}_stack.pdf')
        fstack.quicklook(filename=f'spectra/figures/{basename}_{spwn}_stack.png')


        fig = pl.figure()
        mappable = pl.subplot(1,2,1).imshow(mx.value, origin='lower', interpolation='none')
        pl.colorbar(mappable)
        pl.subplot(1,2,2)
        pl.suptitle(fn)
        sp.quicklook()

In [ ]:
ls spectra

In [ ]:
def line_zooms(globstr, linelist, gridshape=(3,3), vcen=0*u.km/u.s, dv=10*u.km/u.s, ymin=None):

    spectra = [pyspeckit.Spectrum(fn) for fn in glob.glob(globstr)]
    
    fig = pl.gcf()
    
    ax0 = fig.add_subplot(111, frameon=False)
    ax0.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    ax0.set_ylabel("$T_B$ [K]")
    ax0.set_xlabel("$V_{systemic}$ [km/s]")

    plotnum = 1
    for linename, linefreq in linelist.items():
        for sp in spectra:
            if sp.xarr.in_range(linefreq):
                ax = fig.add_subplot(*gridshape, plotnum)
                sp.xarr.convert_to_unit(u.km/u.s, refX=linefreq)
                sp.plotter(axis=ax, xmin=vcen-dv, xmax=vcen+dv, ymin=ymin)
                ax.text(0.99, 0.9, linename, transform=ax.transAxes, ha='right')
                ax.axvline(vcen.value, color='r', linestyle=':', alpha=0.2)
                ax.axhline(0, color='navy', linestyle=':', zorder=-5, alpha=0.3)
                ax.set_xlabel("")
                ax.set_ylabel("")
                sp.xarr.convert_to_unit(u.GHz)
                plotnum += 1
    pl.tight_layout()

pl.figure(figsize=(12,8))
line_zooms('spectra/*disk1*stack.fits', {'NaCl 18-17':  234.251912*u.GHz,
                                         'NaCl 18-17 v=1': 232.509977*u.GHz,
                                         'H$_2$O':  232.6867*u.GHz,
                                         'SiS 12-11':  217.817644*u.GHz,
                                         'SiS 13-12 v=1': 234.8129678*u.GHz,
                                         'NaCl 17-16 v=1': 219.614936*u.GHz,
                                         'PN5-4': 234.935663*u.GHz,
                                         'SiSv=0_13-12': 235.9613634*u.GHz,
                                         'NaClv=2_17-16': 217.979967*u.GHz, # NaCl v=2 17-16
                                         'NaClv=2_18-17':230.7791504*u.GHz, # NaCl v=2 18-17 Unknown_2
                                        }, gridshape=(3,3),
           vcen=0*u.km/u.s,
           dv=30*u.km/u.s,
           ymin=-5)

In [ ]:
stackspectra = []
for ii,fn in enumerate(glob.glob('spectra/*_stack.fits')):
    pysp = pyspeckit.Spectrum(fn)
    basename = os.path.basename(fn)
    spl = basename.split("_")
    name = 'mm1' if spl[0] == 'disk1' else 'mm12' if spl[0] == 'disk3' else 'mm2'
    pysp.specname = f'G351.77 {name} spw{spl[1]} stack'
    stackspectra.append(pysp)

In [ ]:
# changed from vcen=vcen to vcen=0 b/c these are stacked - TODO check this?
overplot_saltlines(stackspectra, vcen=0*u.km/u.s, savepath='spectra/figures', ymax=250, ymin=-25, yfrac=0.55)

rm spectra/figures/color_labels_G351_image_co1_smaller_stack_stack.png

rm spectra/figures/color_labels_G351_54_stack.png

In [ ]:
ls spectra/figures/

In [ ]:
for fn in glob.glob('spectra/figures/color_labels_G351.77_mm[12]_spw[0-3]_stack.png'):
    display(Image(fn))

In [ ]:
for fn in glob.glob('spectra/figures/color_labels_G351.77_mm12_spw[0-3]_stack.png'):
    display(Image(fn))

# Try to ID the lines from Beuther+ that were only tentative

In [ ]:
from astroquery.splatalogue import Splatalogue

In [ ]:
stackspectra

In [ ]:
sp = [x for x in stackspectra if x.xarr.in_range(234.75*u.GHz)][0]
lx,ux = 234.75,234.9
sp.plotter(xmin=lx, xmax=ux)
sp.baseline()
sp.plotter(xmin=lx, xmax=ux)
sp.specfit() #guesses=[5,216.935,0.01], limited=[(True,True),(True,True),(True,True)], limits=[(0,20),(216.93,216.94),(0.0001,0.05)])
sp.baseline(excludefit=True)
sp.plotter(xmin=lx, xmax=ux, ymax=150)
sp.specfit()
cen = sp.specfit.parinfo['SHIFT0']*sp.xarr.unit #* (1 + vcen/constants.c)
dx = (5*u.km/u.s / constants.c)*cen
Splatalogue.query_lines(cen-dx, cen+dx, line_lists=['CDMS'], energy_max=10000, energy_type='eu_k', chemical_name="monosulfide")


In [ ]:
sp = [x for x in stackspectra if x.xarr.in_range(234.9*u.GHz)][0]
lx,ux = 234.9,235
sp.plotter(xmin=lx, xmax=ux)
sp.baseline()
sp.plotter(xmin=lx, xmax=ux)
sp.specfit() #guesses=[5,216.935,0.01], limited=[(True,True),(True,True),(True,True)], limits=[(0,20),(216.93,216.94),(0.0001,0.05)])
sp.baseline(excludefit=True)
sp.plotter(xmin=lx, xmax=ux, ymax=150)
sp.specfit()
cen = sp.specfit.parinfo['SHIFT0']*sp.xarr.unit #* (1 + vcen/constants.c)
dx = (5*u.km/u.s / constants.c)*cen
tbl = Splatalogue.query_lines(cen-dx, cen+dx, energy_max=10000, energy_type='eu_k', chemical_name='PN')
tbl.show_in_notebook(display_length=150)
#cen

In [ ]:
sp = [x for x in stackspectra if x.xarr.in_range(235.9*u.GHz)][0]
lx,ux = 235.9,236.05
sp.plotter(xmin=lx, xmax=ux)
sp.baseline()
sp.plotter(xmin=lx, xmax=ux)
sp.specfit() #guesses=[5,216.935,0.01], limited=[(True,True),(True,True),(True,True)], limits=[(0,20),(216.93,216.94),(0.0001,0.05)])
sp.baseline(excludefit=True)
sp.plotter(xmin=lx, xmax=ux, ymax=150)
sp.specfit()
cen = sp.specfit.parinfo['SHIFT0']*sp.xarr.unit #* (1 + vcen/constants.c)
dx = (5*u.km/u.s / constants.c)*cen
tbl = Splatalogue.query_lines(cen-dx, cen+dx, energy_max=10000, energy_type='eu_k', chemical_name='SiS')
tbl.show_in_notebook(display_length=150)

In [ ]:
Splatalogue.query_lines(215*u.GHz, 236*u.GHz, energy_max=10000, energy_type='eu_k', chemical_name='Silicon monosulfide')

In [ ]:
Splatalogue.query_lines(215*u.GHz, 236*u.GHz, energy_max=10000, energy_type='eu_k', chemical_name='PN')

In [ ]:
import sys
sys.path.append('/orange/adamginsburg/salt/Orion_ALMA_2016.1.00165.S/analysis')
from show_pv import show_pv
import imp
import show_pv
imp.reload(show_pv)
from show_pv import show_keplercurves, show_pv

In [ ]:
import pvextractor

In [ ]:
pathmm1angle = 130*u.deg
pathmm1 = pvextractor.Path(coordinates.SkyCoord([disk1[0].center.directional_offset_by(pathmm1angle,0.15*u.arcsec, ),
                                                 disk1[0].center.directional_offset_by(pathmm1angle-180*u.deg,0.15*u.arcsec, )]))
pvmm1 = pvextractor.extract_pv_slice(stackc, pathmm1)

pathmm1outflow = pvextractor.Path(coordinates.SkyCoord([disk1[0].center.directional_offset_by(pathmm1angle,0.35*u.arcsec, ),
                                                        disk1[0].center.directional_offset_by(pathmm1angle-180*u.deg,0.15*u.arcsec, )]),
                                 width=5)
pvmm1outflow = pvextractor.extract_pv_slice(siocube, pathmm1outflow)

In [ ]:
origin=0.14*u.arcsec
fig,ax,cb,con = show_pv(pvmm1.data, wcs.WCS(pvmm1.header), origin=origin,
                vrange=np.array([vcen.value-30, vcen.value+30]), vcen=vcen,
                        imvmin=np.nanpercentile(pvmm1.data, 3), imvmax=np.nanmax(pvmm1.data),
                        distance=distance,
                        xoffset_label=0.205*u.arcsec
                    )
cb.set_label("K")
kc = show_keplercurves(ax, origin, 500*u.au, 
                       vcen=vcen,
                               masses=[10,20,30,],
                               linestyles=['-']*4,
                               colors=['r','g','b','c'],
                       radii={},
                       distance=distance,
                              )
pl.legend(loc='upper left')
#pl.gca().set_aspect(2.6)
pl.xlabel(f"Offset [\"]  at PA {pathmm1angle} from\n{disk1[0].center.to_string('hmsdms', precision=3)}")

pl.title("G351mm1")
pl.savefig("G351mm1_naclPV_withcolorcurves.png", bbox_inches='tight')
#pl.legend(loc='best')

In [ ]:
origin=0.14*u.arcsec
fig,ax,cb,con = show_pv(pvmm1outflow.data, wcs.WCS(pvmm1outflow.header), origin=origin,
                vrange=np.array([vcen.value-40, vcen.value+40]), vcen=vcen,
                        imvmin=np.nanpercentile(pvmm1outflow.data, 3), imvmax=np.nanmax(pvmm1outflow.data),
                        distance=distance,
                        xoffset_label=0.205*u.arcsec
                    )
cb.set_label("K")
#kc = show_keplercurves(ax, origin, 500*u.au, 
#                       vcen=vcen,
#                               masses=[10,20,30,],
#                               linestyles=['-']*4,
#                               colors=['r','g','b','c'],
#                       radii={},
#                       distance=distance,
#                              )
pl.legend(loc='upper left')
#pl.gca().set_aspect(2.6)
pl.xlabel(f"Offset [\"]  at PA {pathmm1angle} from\n{disk1[0].center.to_string('hmsdms', precision=3)}")

pl.title("G351mm1")
pl.savefig("G351mm1_sioPV_withcolorcurves.png", bbox_inches='tight')
#pl.legend(loc='best')

In [ ]:
pvmm1pn = pvextractor.extract_pv_slice(cubes[1].with_spectral_unit(u.km/u.s, rest_value=234.93569*u.GHz, velocity_convention='radio')
                                       .spectral_slab(vcen-40*u.km/u.s, vcen+40*u.km/u.s),
                                       pathmm1)

In [ ]:
fig,ax,cb,con = show_pv(pvmm1pn.data, wcs.WCS(pvmm1pn.header), origin=origin,
                vrange=np.array([vcen.value-30, vcen.value+30]), vcen=vcen,
                        imvmin=np.nanpercentile(pvmm1pn.data, 3), imvmax=np.nanmax(pvmm1pn.data),
                        distance=distance,
                        xoffset_label=0.205*u.arcsec
                    )
cb.set_label("K")
kc = show_keplercurves(ax, origin, 500*u.au, 
                       vcen=vcen,
                               masses=[10,20,30,],
                               linestyles=['-']*4,
                               colors=['r','g','b','c'],
                       radii={},
                       distance=distance,
                              )
pl.legend(loc='upper left')
pl.xlabel(f"Offset [\"]  at PA {pathmm1angle} from\n{disk1[0].center.to_string('hmsdms', precision=3)}")

pl.title("G351mm1")
pl.savefig("G351mm1_PNPV_withcolorcurves.png", bbox_inches='tight')
#pl.legend(loc='best')

In [ ]:
pvmm1sis = pvextractor.extract_pv_slice(cubes[1].with_spectral_unit(u.km/u.s, rest_value=235.961352*u.GHz, velocity_convention='radio')
                                       .spectral_slab(vcen-40*u.km/u.s, vcen+40*u.km/u.s),
                                       pathmm1)

In [ ]:
pvhdu = pvmm1sis
fig,ax,cb,con = show_pv(pvhdu.data, wcs.WCS(pvhdu.header), origin=origin,
                vrange=np.array([vcen.value-30, vcen.value+30]), vcen=vcen,
                        imvmin=np.nanpercentile(pvhdu.data, 3), imvmax=np.nanmax(pvhdu.data),
                        distance=distance,
                        xoffset_label=0.205*u.arcsec
                    )
cb.set_label("K")
kc = show_keplercurves(ax, origin, 500*u.au, 
                       vcen=vcen,
                               masses=[10,20,30,],
                               linestyles=['-']*4,
                               colors=['r','g','b','c'],
                       radii={},
                       distance=distance,
                              )
pl.legend(loc='upper left')

pl.xlabel(f"Offset [\"]  at PA {pathmm1angle} from\n{disk1[0].center.to_string('hmsdms', precision=3)}")


pl.title("G351mm1")
pl.savefig("G351mm1_SiS1312PV_withcolorcurves.png", bbox_inches='tight')
#pl.legend(loc='best')

In [ ]:
pvmm1h2o = pvextractor.extract_pv_slice(h2ocube, pathmm1)

In [ ]:
pvhdu = pvmm1h2o
fig,ax,cb,con = show_pv(pvhdu.data, wcs.WCS(pvhdu.header), origin=origin,
                vrange=np.array([vcen.value-30, vcen.value+30]), vcen=vcen,
                        imvmin=np.nanpercentile(pvhdu.data, 3), imvmax=np.nanmax(pvhdu.data),
                        distance=distance,
                        xoffset_label=0.205*u.arcsec
                    )
cb.set_label("K")
kc = show_keplercurves(ax, origin, 500*u.au, 
                       vcen=vcen,
                               masses=[10,20,30,],
                               linestyles=['-']*4,
                               colors=['r','g','b','c'],
                       radii={},
                       distance=distance,
                              )
pl.legend(loc='upper left')

pl.xlabel(f"Offset [\"]  at PA {pathmm1angle} from\n{disk1[0].center.to_string('hmsdms', precision=3)}")
ax.set_aspect(4)

pl.title("G351mm1")
pl.savefig("G351mm1_h2oPV_withcolorcurves.png", bbox_inches='tight')
#pl.legend(loc='best')

In [ ]:
m0 = (cubes[1].with_spectral_unit(u.km/u.s, rest_value=235.961352*u.GHz, velocity_convention='radio')
      .spectral_slab(vcen-25*u.km/u.s, vcen+25*u.km/u.s).moment0())

pl.imshow(m0.value, cmap='Blues', origin='lower',)# vmax=m0.value.max()*2,)
          #norm=simple_norm(m0.value, stretch='log', max_cut=m0.value.max()*10))
ax = pl.gca()

dp_60 = (60*u.au / pixscale_au).decompose()

pl.title("G351 mm1")
pl.plot([20, 20+dp_60], [25,25], color='r')
pl.text(20 + dp_60/2, 26, f'60 AU', horizontalalignment='center', color='r')
pixscale = np.abs((wcs.utils.proj_plane_pixel_area(m0.wcs)**0.5*u.deg).to(u.arcsec))
axlims = ax.axis()

xc,yc = 18,18

xtlocs = ax.get_xticks()
xticklabels = map(lambda x: f"{x:0.2f}", (xtlocs-xc)*pixscale.value)
pl.xticks(xtlocs, xticklabels, rotation=45)

ytlocs = ax.get_yticks()
yticklabels = map(lambda x: f"{x:0.2f}", (ytlocs-yc)*pixscale.value)
pl.yticks(ytlocs, yticklabels, rotation=0)

ax.axis(axlims)

bm = stackc.beam
bmell = bm.ellipse_to_plot(5, 4, pixscale)
bmell.set_facecolor('none')
bmell.set_edgecolor('k')
pl.gca().add_artist(bmell)

offset_loc = m0.wcs.celestial.pixel_to_world(xc,yc)


ax.set_xlabel(f"RA offset (arcsec)\nfrom {offset_loc.to_string('hmsdms', precision=3)}")
ax.set_ylabel("Dec offset (arcsec)")
pl.savefig("G351disk1_SiS13-12_mom0_ticks.png", bbox_inches='tight', dpi=300)

In [ ]:
pathmm2angle = -34*u.deg
pathmm2 = pvextractor.Path(coordinates.SkyCoord([disk2[0].center.directional_offset_by(pathmm2angle,0.15*u.arcsec, ),
                                                 disk2[0].center.directional_offset_by(pathmm2angle-180*u.deg,0.15*u.arcsec, )]))
pvmm2 = pvextractor.extract_pv_slice(stack2, pathmm2)

In [ ]:
fig,ax,cb,con = show_pv(pvmm2.data, wcs.WCS(pvmm2.header), origin=0.15*u.arcsec,
                vrange=np.array([vcen.value-30, vcen.value+30]), vcen=vcen,
                        imvmin=np.nanpercentile(pvmm2.data, 3), imvmax=np.nanmax(pvmm2.data),
                        distance=distance,
                        xoffset_label=0.155*u.arcsec
                    )
cb.set_label("K")
kc = show_keplercurves(ax, 0.15*u.arcsec, 500*u.au, 
                       vcen=vcen,
                               masses=[10,20,30,],
                               linestyles=['-']*4,
                               colors=['r','g','b','c'],
                       radii={},
                       distance=distance,
                              )
pl.legend(loc='upper left')

pl.xlabel(f"Offset [\"]  at PA {pathmm2angle} from\n{disk2[0].center.to_string('hmsdms', precision=3)}")


pl.title("G351mm2")
pl.savefig("G351mm2_naclPV_withcolorcurves.png", bbox_inches='tight')
#pl.legend(loc='best')